In [36]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [37]:
!huggingface-cli login


        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/tokens .
        (Deprecated, will be removed in v0.3.0) To login with username and password instead, interrupt with Ctrl+C.
        
Token: 
Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machi

In [38]:
from transformers import BertTokenizer

In [39]:
bt = BertTokenizer.from_pretrained('bert-base-uncased')

In [49]:
text = "[CLS] I like  natural language progressing [SEP]"
tokens = bt.tokenize(text)
tokens[-2] = "progress"

tokens[-1] = "##ing"

tokens.append("[SEP]")
tokens

['[CLS]', 'i', 'like', 'natural', 'language', 'progress', '##ing', '[SEP]']

In [50]:
cand_indexes = []
for i,token in enumerate(tokens):
  if token == "[CLS]" or token == "[SEP]":
    continue
  if len(cand_indexes) >= 1 and token.startswith("##"):
    cand_indexes[-1].append(i)
  else:
    cand_indexes.append([i])

In [54]:
cand_indexes

[[3], [4], [1], [5, 6], [2]]

In [52]:
import random
random.shuffle(cand_indexes)

In [53]:
num_to_predict = max(1,int(round(len(tokens) * 0.15)))
print(num_to_predict)

1


In [55]:
# 采样15%用于mask
masked_lms = []
covered_indexes = set()

for index_set in cand_indexes:
  if len(masked_lms) >= num_to_predict:
    break
  if len(masked_lms) + len(index_set) > num_to_predict:
    break
  is_any_index_covered = False
  for index in index_set:
    if index in covered_indexes:
      is_any_index_covered = True
      break
  if is_any_index_covered:
    continue
  for index in index_set:
    covered_indexes.add(index)
    masked_lms.append(index)

In [ ]:
bt("I like natural language progressing!","rosy if you hear me")

{'input_ids': [101, 1045, 2066, 3019, 2653, 27673, 999, 102, 26851, 2065, 2017, 2963, 2033, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
input_ids = bt.encode("I like natural language progressing!")

In [ ]:
bt.decode(input_ids)

'[CLS] i like natural language progressing! [SEP]'

# DistillBert

In [1]:
from transformers import DistilBertTokenizer, DistilBertModel
import torch

ModuleNotFoundError: ignored

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertModel.from_pretrained("distilbert-base-uncased")

In [ ]:
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")

In [ ]:
inputs

{'input_ids': tensor([[  101,  7592,  1010,  2026,  3899,  2003, 10140,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
outputs = model(**inputs)

In [ ]:
outputs

BaseModelOutput([('last_hidden_state',
                  tensor([[[-1.8296e-01, -7.4054e-02,  5.0268e-02,  ..., -1.1261e-01,
                             4.4493e-01,  4.0941e-01],
                           [ 7.0642e-04,  1.4825e-01,  3.4328e-01,  ..., -8.6040e-02,
                             6.9475e-01,  4.3353e-02],
                           [-5.0721e-01,  5.3086e-01,  3.7163e-01,  ..., -5.6287e-01,
                             1.3756e-01,  2.8475e-01],
                           ...,
                           [-4.2251e-01,  5.7315e-02,  2.4338e-01,  ..., -1.5223e-01,
                             2.4462e-01,  6.4155e-01],
                           [-4.9384e-01, -1.8895e-01,  1.2641e-01,  ...,  6.3240e-02,
                             3.6913e-01, -5.8253e-02],
                           [ 8.3269e-01,  2.4948e-01, -4.5440e-01,  ...,  1.1998e-01,
                            -3.9257e-01, -2.7785e-01]]], grad_fn=<NativeLayerNormBackward0>))])

In [ ]:
inputs = tokenizer("Hello, my dog is cute",'hello world', return_tensors="pt")

In [ ]:
inputs

{'input_ids': tensor([[  101,  7592,  1010,  2026,  3899,  2003, 10140,   102,  7592,  2088,
           102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [34]:
import torch
inputs = ["Hello, my dog is cute","hello world"]

examps = bt(inputs,add_special_tokens=True, truncation=True, max_length=512)
# examps


bt.pad(examps,padding=True,max_length=100)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2302: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  "`max_length` is ignored when `padding`=`True` and there is no truncation strategy. "


{'input_ids': [[101, 7592, 1010, 2026, 3899, 2003, 10140, 102], [101, 7592, 2088, 102, 0, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 0, 0, 0, 0]]}

In [35]:
bt.padding_side

'right'

In [14]:
 [{"input_ids":torch.tensor(e,dtype=torch.long)} for e in examps["input_ids"]]

[{'input_ids': tensor([  101,  7592,  1010,  2026,  3899,  2003, 10140,   102])},
 {'input_ids': tensor([ 101, 7592, 2088,  102])}]

In [24]:
from pprint import pprint

# pprint(vars())

In [25]:
class Runoob:
  a = 1

In [26]:
vars(Runoob)

mappingproxy({'__dict__': <attribute '__dict__' of 'Runoob' objects>,
              '__doc__': None,
              '__module__': '__main__',
              '__weakref__': <attribute '__weakref__' of 'Runoob' objects>,
              'a': 1})